In [1]:
import configparser
import psycopg2
import socket
import sys
import logging
import traceback
import pandas as pd

In [20]:
config = configparser.ConfigParser()
config.read('../config.ini')

# Get the database credentials
db_endpoint = config['redshift']['host']
db_name = config['redshift']['database_name']
db_user = config['redshift']['username']
db_password = config['redshift']['password']
db_port = int(config['redshift']['port'])

# Set the timeout for the connection attempt (in seconds)
connection_timeout = 10

try:    
    # Now establish the database connection using psycopg2
    connection = psycopg2.connect(
        host=db_endpoint,
        port = db_port,
        database = db_name,
        user=db_user,
        password=db_password
    )
    print("Connected successfully!")
    

except (socket.timeout, psycopg2.OperationalError) as e:
    if isinstance(e, socket.timeout):
        print("Error: Connection timed out.")
    else:
        print("Error during connection:", e)
    sys.exit(1)  # Terminate the program with a non-zero exit code

Connected successfully!


In [21]:
with connection.cursor() as cursor:
    cursor.execute("""  SELECT 
                            c.table_schema AS "Esquema",
                            c.table_name AS "Tabla",
                            c.column_name AS "Columna",
                            c.data_type AS "Tipo de Dato",
                            c.character_maximum_length AS "Longitud Máxima",
                            d.description AS "Descripción"
                        FROM 
                            information_schema.columns c
                        LEFT JOIN (
                        SELECT 
                            objoid,
                            objsubid,
                            description
                        FROM 
                            pg_catalog.pg_description
                        ) d ON 
                            d.objoid = (
                            SELECT 
                                oid 
                            FROM pg_class 
                            WHERE relname = c.table_name
                            )
                            AND d.objsubid = (
                            SELECT 
                                ordinal_position 
                            FROM information_schema.columns 
                            WHERE table_name = c.table_name AND column_name = c.column_name
                            )
                        WHERE 
                            c.table_schema NOT IN ('pg_catalog', 'information_schema')
                        ORDER BY 
                            c.table_schema,
                            c.table_name,
                            c.ordinal_position;""")
    columns = [desc[0] for desc in cursor.description]
    data = cursor.fetchall()
    df_trading_historic = pd.DataFrame(data, columns=columns)

In [23]:
df_trading_historic.to_csv('DiccionarioDatosRedshift')